In [1]:
#INSTALL LIBRARIES------------------------------------------
!pip install transformers scikit-learn datasets wandb

In [2]:
#IMPORTS-----------------------------
from pprint import pprint
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaModel
from torch.utils.data import Dataset, DataLoader
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from torch.nn import Linear, ReLU
import pdb
import numpy as np, torch, random as rnd, torch.nn as nn, wandb
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import sys, os, json
from transformers import AutoModelForQuestionAnswering
from torch.nn.functional import cosine_similarity
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

from transformers import AutoTokenizer
import os



In [ ]:


adversarial = load_dataset("iperbole/adversarial_fever_nli")["test"]

ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

training_set = ds["train"]

validation_set = ds["validation"]

test_set = ds["test"]

pprint(adversarial[0])
pprint(training_set[0])

Generating test split:   0%|          | 0/337 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

{'cid': 58846,
 'hypothesis': 'The number of sitcoms from France in which Johnny Galecki has '
               'played a character is greater or equal to 2',
 'label': 'NEUTRAL',
 'part': 'manual_adversarial',
 'premise': 'Johnny Galecki . He is known for playing David Healy in the ABC '
            'sitcom Roseanne from 1992 -- 1997 and Dr. Leonard Hofstadter in '
            'the CBS sitcom The Big Bang Theory since 2007 .'}
{'hypothesis': 'Roman Atwood is a content creator.',
 'id': '150448',
 'label': 'ENTAILMENT',
 'premise': 'Roman Atwood . He is best known for his vlogs , where he posts '
            'updates about his life on a daily basis . His vlogging channel , '
            "`` RomanAtwoodVlogs '' , has a total of 3.3 billion views and "
            '11.9 million subscribers . He also has another YouTube channel '
            "called `` RomanAtwood '' , where he posts pranks .",
 'srl': {'hypothesis': {'annotations': [{'englishPropbank': {'frameName': 'be.01',
              

In [ ]:

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))



['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
f1 = "test per vedere come va"
f2 = "questa è una prova"

tokenized = tokenizer(f1+ tokenizer.eos_token + f2, return_tensors='pt', padding='max_length', max_length=40, return_token_type_ids=True)

print(tokenized)
print(tokenized["input_ids"].shape)
print(type(tokenized))

out = model(**tokenized)
print(out['last_hidden_state'].shape)

print(out.last_hidden_state.mean(dim=-1).squeeze().shape)

{'input_ids': tensor([[    0, 21959,   228,   748,   196,  2816,   283, 13205,     2, 23627,
           102,   952, 11423,   542,   102, 11570,   102,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
torch.Size([1, 40])
<class 'transformers.tokenization_utils_base.BatchEncoding'>
torch.Size([1, 40, 768])


In [3]:
# Function to print a progress bar
def print_progress_bar(percentuale: float, lunghezza_barra: int = 30, text: str="") -> None:
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% complete " + text)
    sys.stdout.flush()

In [23]:
class NLIDataset(Dataset):



    def __init__(self, data, file_name, adversarial = False):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.encode_labels = {'CONTRADICTION': 0, 'NEUTRAL': 1, 'ENTAILMENT': 2}
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.model = RobertaModel.from_pretrained('roberta-base').to(self.device)
        self.distilbert_tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
        self.file_name = file_name
        self.adversarial = adversarial
        self.preprocess_function(data)
        self.tokenizer = None
        self.model = None



    def preprocess_function(self, examples):
        premises = []
        answers = []
        hypothesis = []
        similarities = {}
        ordered_similarities = []
        file_exists = os.path.isfile("data/" + self.file_name)
        if file_exists:
            with open("data/" + self.file_name, "r") as f:
                similarities = json.load(f)
        # Utilizza un ciclo for per popolare le tre liste
        breakpoint()
        for i,q in enumerate(examples):

            print_progress_bar(i / len(examples), text=" | preprocessing")
            premises.append(q["premise"].strip())
            answers.append(self.encode_labels[q["label"]] )
            hypothesis.append(q["hypothesis"].strip())
            if not file_exists:
                s1 = self.embed_sentence(q["premise"].strip())
                s2 = self.embed_sentence(q["hypothesis"].strip())
                if self.adversarial:

                    similarities[q["cid"]] = cosine_similarity(s1, s2).item()
                    ordered_similarities.append(similarities[q["cid"]])
                else:
                    similarities[q["id"]] = cosine_similarity(s1, s2).item()
                    ordered_similarities.append(similarities[q["id"]])
            elif self.adversarial:
                ordered_similarities.append(similarities[q["cid"]])
            else:
                ordered_similarities.append(similarities[q["id"]])

        inputs = self.distilbert_tokenizer(
            hypothesis,
            premises,
            max_length=384,
            truncation="only_second",
            return_offsets_mapping=True,
            padding="max_length",
            return_tensors="pt"
        )
        if not file_exists:
            if not os.path.exists("data"):
                os.makedirs("data")
            with open("data/" + self.file_name, "w") as f:
                json.dump(similarities, f, indent=4)
        inputs["label"] = torch.tensor(answers)
        inputs["similarity"] = torch.tensor(ordered_similarities)
        self.data = inputs


    def embed_sentence(self, sentence):
        # Tokenizza la frase
        inputs = self.tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        # Ottieni gli embedding dal modello
        with torch.no_grad():
            outputs = self.model(**inputs.to(self.device))
        # Usa l'output del modello come embedding (puoi usare altri livelli o combinazioni se preferisci)
        # Prendi il vettore medio (puoi anche scegliere il vettore della [CLS] token, ecc.)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data["input_ids"][idx], self.data["attention_mask"][idx], self.data["label"][idx], self.data["similarity"][idx]

    def collate(self, batch):
        x, attention_mask, y, z = zip(*batch)
        # x = pad_sequence(x, batch_first=True)
        # attention_mask = pad_sequence(attention_mask, batch_first=True)
        return x.to(self.device),attention_mask.to(self.device), y.to(self.device), z.to(self.device)



    def get_dataloader(self, batch_size):
        return DataLoader(self, batch_size=batch_size, shuffle=True, collate_fn = self.collate)

















In [5]:
# prompt: create a torch model using roberta and a linear layer

class RobertaClassifier(nn.Module):
  def __init__(self, use_similarity, num_labels=3):
    super(RobertaClassifier, self).__init__()
    self.distilbert = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")
    self.linear = nn.Linear(self.distilbert.config.hidden_size+1, num_labels)
    self.use_similarity = use_similarity

  def freeze(self, epoch):
    pass



  def forward(self, input_ids, attention_mask, similarities):

    breakpoint()
    if self.use_similarity:
        similarities = torch.zeros(input_ids.shape[0]).to('cuda' if torch.cuda.is_available() else 'cpu')

    outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)

    pooled_output = torch.cat((outputs.pooler_output, similarities.unsqueeze(1)), dim=1) if self.use_similarity else outputs.pooler_output
    # outputs.last_hidden_state.mean(dim=-1)

    logits = self.linear(pooled_output)

    return logits


In [6]:


class Trainer():

    def __init__(self, model,train_dataloader, validation_dataloader, optimizer, loss_function, device, test_dataloader=None):
        self.model = model.to(device)
        self.train_dataloader = train_dataloader
        self.validation_dataloader = validation_dataloader
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.device = device
        self.test_dataloader = test_dataloader


    @staticmethod
    def evaluation_parameters(y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred).ravel()
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        accuracy = accuracy_score(y_true, y_pred)
        return cm, precision, recall, f1, accuracy


    def train(self, epochs: int, use_wandb: bool = False, config: dict = {}, name: str="", target_f1: float=0.0, freezing=[]):
        best_model = None
        save = False
        if use_wandb:
            wandb.init(
                # Set the project where this run will be logged
                project="nlphw2",
                name=name,
                # Track hyperparameters and run metadata
                config=config
            )
        for epoch in range(epochs):
            if epoch in freezing:
                self.model.freeze(epoch)
            self.model.train()  # Set the model to training mode
            total_loss = 0
            #breakpoint()
            for i, batch in enumerate(self.train_dataloader):
                print_progress_bar(i / len(self.train_dataloader), text=f" | training epoch {epoch}")
                # Get the inputs and targets from the batch
                inputs, mask, targets, similarities = batch

                # Zero the gradients
                self.optimizer.zero_grad()
                # Forward pass
                outputs = self.model(inputs, mask, similarities)
                #print("outputs = ", outputs,"\ntargets = ", targets)
                #breakpoint()
                # Compute loss
                loss = self.loss_function(outputs, targets)
                #print(loss)
                # Backward pass and optimize
                loss.backward()
                self.optimizer.step()
                # Accumulate the total loss
                total_loss += loss.item()

            # Print the average loss for this epoch
            validation_loss, precision, recall, f1, accuracy = self.validate(use_wandb)
            if f1 > target_f1:
                best_model = self.model.state_dict()
                target_f1 = f1
                save = True
            if use_wandb:
                wandb.log({"validation_loss": validation_loss,
                      "precision": precision,
                      "recall": recall,
                      "f1": f1,
                      "accuracy": accuracy,
                      "train_loss": total_loss / len(self.train_dataloader)})
        if save:
            torch.save(best_model, name + f'-{target_f1}.pth')
        print(target_f1)
        if use_wandb:
            wandb.finish()



    def validate(self, use_wandb: bool = False, test=False):
        dataloader = self.test_dataloader if test else self.validation_dataloader
        if dataloader is None:
            print("empty dataloader!")
            exit(1)
        self.model.eval()  # Set the model to evaluation mode
        total_loss = 0
        all_predictions = torch.tensor([])
        all_targets = torch.tensor([])
        with torch.no_grad():  # Do not calculate gradients
            for i, batch in enumerate(self.validation_dataloader):
                print_progress_bar(i / len(dataloader), text=" | validation")
                # Get the inputs and targets from the batch
                inputs, mask, targets, similarities  = batch

                # Forward pass
                outputs = self.model(inputs, mask, similarities)
                # Compute loss
                #breakpoint()
                loss = self.loss_function(outputs, targets)
                # Accumulate the total loss
                total_loss += loss.item()
                # Store predictions and targets
                all_predictions = torch.cat((all_predictions, outputs.squeeze().round().cpu()))
                all_targets = torch.cat((all_targets, targets.cpu()))
        validation_loss = total_loss / len(self.validation_dataloader)
        #breakpoint()
        precision = precision_score(all_targets, all_predictions)
        recall = recall_score(all_targets, all_predictions)
        f1 = f1_score(all_targets, all_predictions)
        accuracy = accuracy_score(all_targets, all_predictions)
        return validation_loss, precision, recall, f1, accuracy




In [7]:

adversarial = load_dataset("iperbole/adversarial_fever_nli")["test"]

ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

training_set = ds["train"]

validation_set = ds["validation"]

test_set = ds["test"]

wandb.login(key='aaf831dabc88d936d4e6b439b798bb4cb42814ea')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/337 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
!git clone https://github.com/monteleone-1883922/hw2_nlp.git
os.chdir("hw2_nlp")
!git checkout huggingFaceBase
!mv data ./../data
os.chdir("..")
!rm -rf hw2_nlp


Cloning into 'hw2_nlp'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 44 (delta 22), reused 11 (delta 1), pack-reused 0
Receiving objects: 100% (44/44), 660.50 KiB | 3.57 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Branch 'huggingFaceBase' set up to track remote branch 'huggingFaceBase' from 'origin'.
Switched to a new branch 'huggingFaceBase'


In [25]:
train_dataset = NLIDataset(training_set, "train_similarities.json")

validation_dataset = NLIDataset(validation_set, "validation_similarities.json")

test_dataset = NLIDataset(test_set, "test_similarities.json")

adversarial_dataset = NLIDataset(adversarial, "adversarial_similarities.json", adversarial=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


> <ipython-input-23-33f360525cf6>(31)preprocess_function()
     29         # Utilizza un ciclo for per popolare le tre liste
     30         breakpoint()
---> 31         for i,q in enumerate(examples):
     32 
     33             print_progress_bar(i / len(examples), text=" | preprocessing")

ipdb> q


In [26]:
adversarial_dataset = NLIDataset(adversarial, "adversarial_similarities.json", adversarial=True)
with open("data/" + "adversarial_similarities.json" , "r") as f:
    similarities = json.load(f)
print(len(similarities))
print(len(adversarial))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


> <ipython-input-23-33f360525cf6>(31)preprocess_function()
     29         # Utilizza un ciclo for per popolare le tre liste
     30         breakpoint()
---> 31         for i,q in enumerate(examples):
     32 
     33             print_progress_bar(i / len(examples), text=" | preprocessing")

ipdb> n
> <ipython-input-23-33f360525cf6>(33)preprocess_function()
     31         for i,q in enumerate(examples):
     32 
---> 33             print_progress_bar(i / len(examples), text=" | preprocessing")
     34             premises.append(q["premise"].strip())
     35             answers.append(self.encode_labels[q["label"]] )

ipdb> q["cid"]


In [18]:
for ex in adversarial:
    if ex["cid"] not in similarities:
        print(ex)

{'cid': 58846,
 'hypothesis': 'The number of sitcoms from France in which Johnny Galecki has '
               'played a character is greater or equal to 2',
 'label': 'NEUTRAL',
 'part': 'manual_adversarial',
 'premise': 'Johnny Galecki . He is known for playing David Healy in the ABC '
            'sitcom Roseanne from 1992 -- 1997 and Dr. Leonard Hofstadter in '
            'the CBS sitcom The Big Bang Theory since 2007 .'}


In [ ]:
# prompt: genera i dataloader

train_dataloader = train_dataset.get_dataloader(batch_size=32)
validation_dataloader = validation_dataset.get_dataloader(batch_size=32)
test_dataloader = test_dataset.get_dataloader(batch_size=32)
adversarial_dataloader = adversarial_dataset.get_dataloader(batch_size=32)


In [ ]:

model = RobertaClassifier(False)

trainer = Trainer(model, train_dataloader, validation_dataloader, torch.optim.Adam(model.parameters(), lr=1e-4), nn.CrossEntropyLoss(),'cuda' if torch.cuda.is_available() else 'cpu', test_dataloader)



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

trainer.train(20, use_wandb=True, name="first")




wandb: Currently logged in as: monteleone-1883922 (monteleone). Use `wandb login --relogin` to force relogin


[>                              ] 2.62% complete  | training epoch 0

IndexError: index out of range in self

In [ ]:
del training_set

dataset = NLIDataset(validation_set)


dataloader = dataset.get_dataloader(batch_size=32, pos_num=1)

model = RobertaClassifier()

batch = next(iter(dataloader))

model(batch[0], batch[1], batch[3])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 336, in set_trace
    sys.settrace(self.trace_dispa

> <ipython-input-5-fac945463f68>(13)forward()
     11 
     12     breakpoint()
---> 13     with torch.no_grad():
     14       outputs = self.roberta_similarity(input_ids=input_ids, attention_mask=attention_mask)
     15       premise = outputs.last_hidden_state[:,0, :]

ipdb> n
> <ipython-input-5-fac945463f68>(14)forward()
     12     breakpoint()
     13     with torch.no_grad():
---> 14       outputs = self.roberta_similarity(input_ids=input_ids, attention_mask=attention_mask)
     15       premise = outputs.last_hidden_state[:,0, :]
     16       hypotehsis = outputs.last_hidden_state[torch.arange(outputs.last_hidden_state.size(0)),separators, :]

ipdb> n
> <ipython-input-5-fac945463f68>(15)forward()
     13     with torch.no_grad():
     14       outputs = self.roberta_similarity(input_ids=input_ids, attention_mask=attention_mask)
---> 15       premise = outputs.last_hidden_state[:,0, :]
     16       hypotehsis = outputs.last_hidden_state[torch.arange(outputs.last_hidden_state.s


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 361, in set_quit
    sys.settrace(None)

